In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets.activation_dataset import ActivationDataset
from copy import deepcopy
import os
import torch.optim as optim
from tqdm.auto import tqdm  # For progress bar
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16,apply=True):
        super(SEBlock, self).__init__()
        self.apply = apply
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x) if self.apply else y.expand_as(x)



class AutoencoderWithSE(nn.Module):
    def __init__(self, in_channels, latent_channels, reduction=16):
        super(AutoencoderWithSE, self).__init__()
        self.se_block = SEBlock(in_channels, reduction)
        self.encoder = nn.Conv2d(in_channels, latent_channels, kernel_size=1)
        self.decoder = nn.Conv2d(latent_channels, in_channels, kernel_size=1)

    def forward(self, x):
        # Apply Squeeze-and-Excite block before encoding
        x = self.se_block(x)
        # Encoding with 1x1 convolution
        encoded = self.encoder(x)
        # Decoding with 1x1 convolution
        decoded = self.decoder(encoded)
        return encoded, decoded

# Example usage:
if __name__ == "__main__":
    # Assuming input is a batch of images with 3 channels (e.g., RGB)
    act= {
    'config': {
      'root_dir': "/media/wmg-5gcat/Co-op Autonomy 2/Hakan/custom_dataset/nus_centerpoint_activations_filtered_objects_lonly/"  ,
      'label_file': 'nus_centerpoint_labels_filtered_objects_lonly.csv',
      'classes': ['No Error', 'Error'],
      'label_field': 'is_missed',
      'layer': [0,1,2],
      'is_multi_feature': True,
      'is_sparse': True,
      'extension':'.pt',
      'name': 'nuscenes'
      }
      }
    act_train = deepcopy(act)
    act_train['config']['root_dir'] = os.path.join(act['config']['root_dir'], 'train')
    act_train_dataset = ActivationDataset(act_train['config'])
    act_val = deepcopy(act)
    act_val['config']['root_dir'] = os.path.join(act['config']['root_dir'], 'val')
    act_val_dataset = ActivationDataset(act_val['config'])
    act_test = deepcopy(act)
    act_test['config']['root_dir'] = os.path.join(act['config']['root_dir'], 'test')
    act_test_dataset = ActivationDataset(act_test['config'])
    
    train_loader = torch.utils.data.DataLoader(act_train_dataset, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(act_val_dataset, batch_size=32, shuffle=False)
    test_loader = torch.utils.data.DataLoader(act_test_dataset, batch_size=1, shuffle=False)
    
    # Define the autoencoder with Squeeze-and-Excite
    model = AutoencoderWithSE(in_channels=256, latent_channels=1)  # Compressing to 16 channels
    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Training loop
    num_epochs = 20  # Number of epochs
    model.cuda()  # Move model to GPU if available
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        train_loss = 0.0
        
        # Training phase
        for data,_,_ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs = data[0] # Move data to GPU if available
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Forward pass
            _, outputs = model(inputs)
            
            # Compute the loss
            loss = mse_loss(outputs, inputs)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            # Accumulate the training loss
            train_loss += loss.item() * inputs.size(0)
        
        # Calculate average loss over the training data
        train_loss /= len(train_loader.dataset)
        
        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        
        with torch.no_grad():
            for data,_,_ in val_loader:
                print(len(data))
                inputs = data[0]  # Move data to GPU if available
                
                # Forward pass
                _, outputs = model(inputs)
                
                # Compute the loss
                loss = mse_loss(outputs, inputs)
                
                # Accumulate the validation loss
                val_loss += loss.item() * inputs.size(0)
        
        # Calculate average loss over the validation data
        val_loss /= len(val_loader.dataset)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    # After training, evaluate on the test set if needed
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0

    with torch.no_grad():
        for data,_,_ in test_loader:
            inputs = data[0] # Move data to GPU if available
            
            # Forward pass
            _, outputs = model(inputs)
            
            # Compute the loss
            loss = mse_loss(outputs, inputs)
            
            # Accumulate the test loss
            test_loss += loss.item() * inputs.size(0)

    # Calculate average loss over the test data
    test_loss /= len(test_loader.dataset)

    print(f"Test Loss: {test_loss:.4f}")
        # Forward pass

        # encoded, decoded = model(tens[0].unsqueeze(0).cpu())
        
        # print("Encoded shape:", encoded.shape)
        # print("Decoded shape:", decoded.shape)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
ROOT DIR:  /media/wmg-5gcat/Co-op Autonomy 2/Hakan/custom_dataset/nus_centerpoint_activations_filtered_objects_lonly/train
Number of features found:  22551  in  ['/media/wmg-5gcat/Co-op Autonomy 2/Hakan/custom_dataset/nus_centerpoint_activations_filtered_objects_lonly/train/features/n008-2018-05-21-11-06-59-0400__LIDAR_TOP__1526915243047392.pt', '/media/wmg-5gcat/Co-op Autonomy 2/Hakan/custom_dataset/nus_centerpoint_activations_filtered_objects_lonly/train/features/n008-2018-05-21-11-06-59-0400__LIDAR_TOP__1526915243547836.pt', '/media/wmg-5gcat/Co-op Autonomy 2/Hakan/custom_dataset/nus_centerpoint_activations_filtered_objects_lonly/train/features/n008-2018-05-21-11-06-59-0400__LIDAR_TOP__1526915244048280.pt', '/media/wmg-5gcat/Co-op Autonomy 2/Hakan/custom_dataset/nus_centerpoint_activations_filtered_objects_lonly/trai

Epoch 1/20:   0%|          | 0/705 [00:00<?, ?it/s]

KeyboardInterrupt: 